In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from numpy.random import randn
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.cross_validation import KFold;
from sklearn import preprocessing
import math
from sklearn.metrics import mean_squared_error


## Logistic -- linear model

### Train the model

In [2]:
train = pd.read_csv("train.csv")

In [3]:
train["area"] = train["slotheight"]*train["slotwidth"] 

feature = ["weekday","hour","advertiser","adexchange","area","slotvisibility","slotformat",
          "slotprice"]
X_train = train[feature]
Y_train = train["click"]
X_train.head()

,weekday,hour,advertiser,adexchange,area,slotvisibility,slotformat,slotprice
0,1,14,1458,2,28080,1,0,5
1,2,20,3476,2,62500,1,0,5
2,5,9,1458,1,94080,0,1,0
3,2,20,3427,2,65520,0,0,162
4,6,9,3476,1,85500,0,1,0


In [4]:
X_train.loc[ X_train['slotprice'] <= 0, 'slotprice'] = 0
X_train.loc[(X_train['slotprice'] > 0) & (X_train['slotprice'] <= 10), 'slotprice'] = 1
X_train.loc[(X_train['slotprice'] > 10) & (X_train['slotprice'] <= 50), 'slotprice'] = 2
X_train.loc[(X_train['slotprice'] > 50) & (X_train['slotprice'] <= 100), 'slotprice'] = 3
X_train.loc[(X_train['slotprice'] > 100), 'slotprice'] = 4


/Users/Wendy/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [5]:
X_train.area.value_counts().sort_index()
X_train.loc[ X_train['area'] <= 20000, 'area'] = 0
X_train.loc[(X_train['area'] > 20000) & (X_train['area'] <=40000 ), 'area'] = 1
X_train.loc[(X_train['area'] > 40000) & (X_train['area'] <= 60000), 'area'] = 2
X_train.loc[(X_train['area'] > 60000) & (X_train['area'] <= 80000), 'area'] = 3
X_train.loc[(X_train['area'] > 80000) & (X_train['area'] <= 100000), 'area'] = 4
X_train.loc[(X_train['area'] > 100000), 'area'] = 5


/Users/Wendy/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [6]:
X_train.head()

,weekday,hour,advertiser,adexchange,area,slotvisibility,slotformat,slotprice
0,1,14,1458,2,1,1,0,1
1,2,20,3476,2,3,1,0,1
2,5,9,1458,1,4,0,1,0
3,2,20,3427,2,3,0,0,4
4,6,9,3476,1,4,0,1,0


In [7]:
X_train["advertiser"]=X_train.advertiser.astype("str")
X_train["weekday"]=X_train.weekday.astype("str")
X_train["adexchange"]=X_train.adexchange.astype("str")
X_train["slotvisibility"]=X_train.slotvisibility.astype("str")
X_train["slotformat"]=X_train.slotformat.astype("str")
X_train["hour"]=X_train.hour.astype("str")


X_train=pd.get_dummies(X_train)

/Users/Wendy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/Wendy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/Wendy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [8]:
logreg = LogisticRegression(penalty='l2', dual=False, tol=0.0001)
logreg.fit(X_train, Y_train)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

99.920000000000002

In [9]:
Y_pred = logreg.predict_proba(X_train)
Y_pred_log = Y_pred[:,1]
Y_pred_log

array([ 0.00079748,  0.0006675 ,  0.00086295, ...,  0.000347  ,
        0.00071354,  0.00091049])

In [10]:
import math
from sklearn.metrics import mean_squared_error
MSE = mean_squared_error(Y_pred_log ,Y_train)
RMSE = math.sqrt(MSE)
print("RMSE: " + str(RMSE) )

RMSE: 0.027433795066621594


### Validation

In [11]:
validation = pd.read_csv("validation.csv")

In [12]:
validation["area"] = validation["slotheight"]*validation["slotwidth"] 

feature = ["weekday","hour","advertiser","adexchange","area","slotvisibility","slotformat",
          "slotprice"]
X_validation = validation[feature]
Y_validation = validation["click"]
X_validation.head()

,weekday,hour,advertiser,adexchange,area,slotvisibility,slotformat,slotprice
0,5,14,3427,1,75000,0,1,0
1,3,14,2997,null,16000,FirstView,Na,118
2,1,13,3427,3,90000,0,0,70
3,5,18,3427,3,90000,0,0,70
4,5,14,3386,2,94080,2,0,5


In [13]:
X_validation.loc[ X_validation['slotprice'] <= 0, 'slotprice'] = 0
X_validation.loc[(X_validation['slotprice'] > 0) & (X_validation['slotprice'] <= 10), 'slotprice'] = 1
X_validation.loc[(X_validation['slotprice'] > 10) & (X_validation['slotprice'] <= 50), 'slotprice'] = 2
X_validation.loc[(X_validation['slotprice'] > 50) & (X_validation['slotprice'] <= 100), 'slotprice'] = 3
X_validation.loc[(X_validation['slotprice'] > 100), 'slotprice'] = 4

X_validation.area.value_counts().sort_index()
X_validation.loc[ X_validation['area'] <= 20000, 'area'] = 0
X_validation.loc[(X_validation['area'] > 20000) & (X_validation['area'] <=40000 ), 'area'] = 1
X_validation.loc[(X_validation['area'] > 40000) & (X_validation['area'] <= 60000), 'area'] = 2
X_validation.loc[(X_validation['area'] > 60000) & (X_validation['area'] <= 80000), 'area'] = 3
X_validation.loc[(X_validation['area'] > 80000) & (X_validation['area'] <= 100000), 'area'] = 4
X_validation.loc[(X_validation['area'] > 100000), 'area'] = 5


/Users/Wendy/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [14]:
X_validation["advertiser"]=X_validation.advertiser.astype("str")
X_validation["weekday"]=X_validation.weekday.astype("str")
X_validation["adexchange"]=X_validation.adexchange.astype("str")
X_validation["slotvisibility"]=X_validation.slotvisibility.astype("str")
X_validation["slotformat"]=X_validation.slotformat.astype("str")
X_validation["hour"]=X_validation.hour.astype("str")

# X_validation["slotprice"]=X_validation.slotprice.astype("str")
# X_validation["area"]=X_validation.area.astype("str")

X_validation=pd.get_dummies(X_validation)

/Users/Wendy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/Wendy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/Wendy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [15]:
Y_pred_validation = logreg.predict_proba(X_validation)
Y_pred_validation_log = Y_pred_validation[:,1]
Y_pred_validation_log

array([ 0.00063213,  0.005724  ,  0.00085562, ...,  0.0003533 ,
        0.00025509,  0.00114593])

In [16]:
MSE = mean_squared_error(Y_pred_validation_log,Y_validation)
RMSE = math.sqrt(MSE)
print("RMSE: " + str(RMSE) )

RMSE: 0.027433625483877472


In [17]:
validation["pCTR_log"] = Y_pred_validation_log

# avgCTR = (train[train.click==1].groupby("advertiser").size().values/train.groupby("advertiser").size().values).mean()
avgCTR = Y_pred_validation_log.mean()

param_validation_log= Y_pred_validation_log/avgCTR
param_validation_log

array([ 0.84095098,  7.61487083,  1.13825908, ...,  0.4700084 ,
        0.33935925,  1.52447069])

#### base bid = 70

In [18]:
validation['new_bidprice_log'] = 70*param_validation_log

statistic=pd.DataFrame()
new_validation = validation[validation["new_bidprice_log"]>=validation.payprice]
new_validation["total_cost"] = new_validation["payprice"].cumsum()
budget_1 = new_validation[new_validation["total_cost"]/1000<=25000]
statistic["Click"] = [budget_1[budget_1.click==1].click.sum()]
statistic["Spend"] = budget_1.payprice.sum()/1000
statistic["CTR%"] = statistic["Click"]/budget_1.shape[0]*100
statistic["CPC"] = statistic["Spend"]/statistic["Click"]
statistic["CPM"] = statistic["Spend"] /budget_1.shape[0]* 1000

statistic_2=pd.DataFrame()
budget_2 = new_validation[new_validation["total_cost"]/1000<=25000*1/2]
statistic_2["Click"] = [budget_2[budget_2.click==1].click.sum()]
statistic_2["Spend"] = budget_2.payprice.sum()/1000
statistic_2["CTR%"] = statistic_2["Click"]/budget_2.shape[0]*100
statistic_2["CPC"] = statistic_2["Spend"]/statistic_2["Click"]
statistic_2["CPM"] = statistic_2["Spend"] /budget_2.shape[0]* 1000

statistic_4=pd.DataFrame()
budget_4 = new_validation[new_validation["total_cost"]/1000<=25000/4]
statistic_4["Click"] = [budget_4[budget_4.click==1].click.sum()]
statistic_4["Spend"] = budget_4.payprice.sum()/1000
statistic_4["CTR%"] = statistic_4["Click"]/budget_4.shape[0]*100
statistic_4["CPC"] = statistic_4["Spend"]/statistic_4["Click"]
statistic_4["CPM"] = statistic_4["Spend"] /budget_4.shape[0]* 1000

statistic_8=pd.DataFrame()
budget_8 = new_validation[new_validation["total_cost"]/1000<=25000*1/8]
statistic_8["Click"] = [budget_8[budget_8.click==1].click.sum()]
statistic_8["Spend"] = budget_8.payprice.sum()/1000
statistic_8["CTR%"] = statistic_8["Click"]/budget_8.shape[0]*100
statistic_8["CPC"] = statistic_8["Spend"]/statistic_8["Click"]
statistic_8["CPM"] = statistic_8["Spend"] /budget_8.shape[0]* 1000

statistic_16=pd.DataFrame()
budget_16 = new_validation[new_validation["total_cost"]/1000<=25000*1/16]
statistic_16["Click"] = [budget_16[budget_16.click==1].click.sum()]
statistic_16["Spend"] = budget_16.payprice.sum()/1000
statistic_16["CTR%"] = statistic_16["Click"]/budget_16.shape[0]*100
statistic_16["CPC"] = statistic_16["Spend"]/statistic_16["Click"]
statistic_16["CPM"] = statistic_16["Spend"] /budget_16.shape[0]* 1000


statistic_one = statistic.append(statistic_2)
statistic_two = statistic_one.append(statistic_4)
statistic_3 = statistic_two.append(statistic_8)
statistic_full_val_log = statistic_3.append(statistic_16)

statistic_full_val_log.index = ["full","1/2","1/4 (6250)","1/8","1/16"]
statistic_full_val_log

/Users/Wendy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Click,Spend,CTR%,CPC,CPM
full,99,5747.242,0.070494,58.052949,40.923696
1/2,99,5747.242,0.070494,58.052949,40.923696
1/4 (6250),99,5747.242,0.070494,58.052949,40.923696
1/8,57,3124.960,0.074668,54.823860,40.935838
1/16,34,1562.478,0.088731,45.955235,40.776606


## Gradient

In [19]:
train["area"] = train["slotheight"]*train["slotwidth"] 

city = pd.DataFrame(train.city.value_counts()/train.shape[0]).index
frequent = pd.DataFrame(train.city.value_counts()/train.shape[0]).values
r = pd.DataFrame()
r["city"] = city
r["frequent"] = frequent
train=pd.merge(train,r,on='city',how='left') 

feature = ["weekday","hour","advertiser","adexchange","area","slotvisibility","slotformat","frequent",
          "slotprice"]
X_train = train[feature]
Y_train = train["click"]

X_train.head()

,weekday,hour,advertiser,adexchange,area,slotvisibility,slotformat,frequent,slotprice
0,1,14,1458,2,28080,1,0,0.019417,5
1,2,20,3476,2,62500,1,0,0.013130,5
2,5,9,1458,1,94080,0,1,0.039022,0
3,2,20,3427,2,65520,0,0,0.004477,162
4,6,9,3476,1,85500,0,1,0.001714,0


In [20]:
np.random.seed(12)

# Initialize label encoder
label_encoder = preprocessing.LabelEncoder()

X_train["slotvisibility"] = label_encoder.fit_transform(X_train["slotvisibility"])
X_train["slotformat"] = label_encoder.fit_transform(X_train["slotformat"])
X_train["adexchange"] = label_encoder.fit_transform(X_train["adexchange"])

X_train["advertiser"]=X_train.advertiser.astype("str")
X_train["weekday"]=X_train.weekday.astype("str")
X_train["hour"]=X_train.hour.astype("str")

X_train=pd.get_dummies(X_train)

/Users/Wendy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Wendy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Wendy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/in

In [21]:
GBR = GradientBoostingClassifier()
GBR.fit(X_train, Y_train)

GradientBoostingClassifier(init=None, learning_rate=0.1, loss='deviance',
              max_depth=3, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [22]:
Y_pred_ = GBR.predict_proba(X_train)
Y_pred_gbr = Y_pred_[:,1]
Y_pred_gbr

array([ 0.00076917,  0.00041292,  0.00108197, ...,  0.00024882,
        0.00047198,  0.00082217])

In [23]:
MSE = mean_squared_error(Y_pred_gbr,Y_train)
RMSE = math.sqrt(MSE)
print("RMSE: " + str(RMSE) )

RMSE: 0.027383548816209902


### Validation

In [24]:
validation["area"] = validation["slotheight"]*validation["slotwidth"] 

city = pd.DataFrame(validation.city.value_counts()/validation.shape[0]).index
frequent = pd.DataFrame(validation.city.value_counts()/validation.shape[0]).values
r = pd.DataFrame()
r["city"] = city
r["frequent"] = frequent
validation=pd.merge(validation,r,on='city',how='left') 

feature = ["weekday","hour","advertiser","adexchange","area","slotvisibility","slotformat","frequent",
          "slotprice"]
X_validation = validation[feature]
Y_validation = validation["click"]

X_validation.head()

,weekday,hour,advertiser,adexchange,area,slotvisibility,slotformat,frequent,slotprice
0,5,14,3427,1,75000,0,1,0.002068,0
1,3,14,2997,null,16000,FirstView,Na,0.039229,118
2,1,13,3427,3,90000,0,0,0.003676,70
3,5,18,3427,3,90000,0,0,0.010175,70
4,5,14,3386,2,94080,2,0,0.021628,5


In [25]:
np.random.seed(12)

# Initialize label encoder
label_encoder = preprocessing.LabelEncoder()

X_validation["slotvisibility"] = label_encoder.fit_transform(X_validation["slotvisibility"])
X_validation["slotformat"] = label_encoder.fit_transform(X_validation["slotformat"])
X_validation["adexchange"] = label_encoder.fit_transform(X_validation["adexchange"])


X_validation["advertiser"]=X_validation.advertiser.astype("str")
X_validation["weekday"]=X_validation.weekday.astype("str")
X_validation["hour"]=X_validation.hour.astype("str")

X_validation=pd.get_dummies(X_validation)

/Users/Wendy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Wendy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Wendy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/in

In [26]:
Y_pred_validation_gbr = GBR.predict_proba(X_validation)
Y_pred_validation_gbr = Y_pred_validation_gbr [:,1]
MSE = mean_squared_error(Y_pred_validation_gbr,Y_validation)
RMSE = math.sqrt(MSE)
print("RMSE: " + str(RMSE) )

RMSE: 0.02751658128834068


#### base price = 70

In [27]:
validation["pCTR_gbr"] = Y_pred_validation_gbr

avgCTR = Y_pred_validation_gbr.mean()

param_validation_gbr= Y_pred_validation_gbr/avgCTR

validation['new_bidprice_gbr'] = 70*param_validation_gbr

statistic=pd.DataFrame()
new_validation = validation[validation["new_bidprice_gbr"]>=validation.payprice]
new_validation["total_cost"] = new_validation["payprice"].cumsum()
budget_1 = new_validation[new_validation["total_cost"]/1000<=25000]
statistic["Click"] = [budget_1[budget_1.click==1].click.sum()]
statistic["Spend"] = budget_1.payprice.sum()/1000
statistic["CTR%"] = statistic["Click"]/budget_1.shape[0]*100
statistic["CPC"] = statistic["Spend"]/statistic["Click"]
statistic["CPM"] = statistic["Spend"] /budget_1.shape[0]* 1000

statistic_2=pd.DataFrame()
budget_2 = new_validation[new_validation["total_cost"]/1000<=25000*1/2]
statistic_2["Click"] = [budget_2[budget_2.click==1].click.sum()]
statistic_2["Spend"] = budget_2.payprice.sum()/1000
statistic_2["CTR%"] = statistic_2["Click"]/budget_2.shape[0]*100
statistic_2["CPC"] = statistic_2["Spend"]/statistic_2["Click"]
statistic_2["CPM"] = statistic_2["Spend"] /budget_2.shape[0]* 1000

statistic_4=pd.DataFrame()
budget_4 = new_validation[new_validation["total_cost"]/1000<=25000/4]
statistic_4["Click"] = [budget_4[budget_4.click==1].click.sum()]
statistic_4["Spend"] = budget_4.payprice.sum()/1000
statistic_4["CTR%"] = statistic_4["Click"]/budget_4.shape[0]*100
statistic_4["CPC"] = statistic_4["Spend"]/statistic_4["Click"]
statistic_4["CPM"] = statistic_4["Spend"] /budget_4.shape[0]* 1000

statistic_8=pd.DataFrame()
budget_8 = new_validation[new_validation["total_cost"]/1000<=25000*1/8]
statistic_8["Click"] = [budget_8[budget_8.click==1].click.sum()]
statistic_8["Spend"] = budget_8.payprice.sum()/1000
statistic_8["CTR%"] = statistic_8["Click"]/budget_8.shape[0]*100
statistic_8["CPC"] = statistic_8["Spend"]/statistic_8["Click"]
statistic_8["CPM"] = statistic_8["Spend"] /budget_8.shape[0]* 1000

statistic_16=pd.DataFrame()
budget_16 = new_validation[new_validation["total_cost"]/1000<=25000*1/16]
statistic_16["Click"] = [budget_16[budget_16.click==1].click.sum()]
statistic_16["Spend"] = budget_16.payprice.sum()/1000
statistic_16["CTR%"] = statistic_16["Click"]/budget_16.shape[0]*100
statistic_16["CPC"] = statistic_16["Spend"]/statistic_16["Click"]
statistic_16["CPM"] = statistic_16["Spend"] /budget_16.shape[0]* 1000


statistic_one = statistic.append(statistic_2)
statistic_two = statistic_one.append(statistic_4)
statistic_3 = statistic_two.append(statistic_8)
statistic_full_val_grb = statistic_3.append(statistic_16)

statistic_full_val_grb.index = ["full","1/2","1/4 (6250)","1/8","1/16"]
statistic_full_val_grb

/Users/Wendy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Click,Spend,CTR%,CPC,CPM
full,117,6039.890,0.081794,51.622991,42.224296
1/2,117,6039.890,0.081794,51.622991,42.224296
1/4 (6250),117,6039.890,0.081794,51.622991,42.224296
1/8,64,3124.878,0.086241,48.826219,42.108016
1/16,37,1562.369,0.099433,42.226189,41.986751


### Combination

In [28]:
Y_pred_validation_combine = 0.1 * Y_pred_validation_log + 0.9 * Y_pred_validation_gbr
validation["pCTR_combine"] = Y_pred_validation_combine
avg_ctr = Y_pred_validation_combine.mean()
param_validation_combine = Y_pred_validation_combine/avg_ctr

validation["new_bidprice_combine"] = 70 * param_validation_combine

statistic=pd.DataFrame()
new_validation = validation[validation["new_bidprice_combine"]>=validation.payprice]
new_validation["total_cost"] = new_validation["payprice"].cumsum()
budget_1 = new_validation[new_validation["total_cost"]/1000<=25000]
statistic["Click"] = [budget_1[budget_1.click==1].click.sum()]
statistic["Spend"] = budget_1.payprice.sum()/1000
statistic["CTR%"] = statistic["Click"]/budget_1.shape[0]*100
statistic["CPC"] = statistic["Spend"]/statistic["Click"]
statistic["CPM"] = statistic["Spend"] /budget_1.shape[0]* 1000

statistic_2=pd.DataFrame()
budget_2 = new_validation[new_validation["total_cost"]/1000<=25000*1/2]
statistic_2["Click"] = [budget_2[budget_2.click==1].click.sum()]
statistic_2["Spend"] = budget_2.payprice.sum()/1000
statistic_2["CTR%"] = statistic_2["Click"]/budget_2.shape[0]*100
statistic_2["CPC"] = statistic_2["Spend"]/statistic_2["Click"]
statistic_2["CPM"] = statistic_2["Spend"] /budget_2.shape[0]* 1000

statistic_4=pd.DataFrame()
budget_4 = new_validation[new_validation["total_cost"]/1000<=25000/4]
statistic_4["Click"] = [budget_4[budget_4.click==1].click.sum()]
statistic_4["Spend"] = budget_4.payprice.sum()/1000
statistic_4["CTR%"] = statistic_4["Click"]/budget_4.shape[0]*100
statistic_4["CPC"] = statistic_4["Spend"]/statistic_4["Click"]
statistic_4["CPM"] = statistic_4["Spend"] /budget_4.shape[0]* 1000

statistic_8=pd.DataFrame()
budget_8 = new_validation[new_validation["total_cost"]/1000<=25000*1/8]
statistic_8["Click"] = [budget_8[budget_8.click==1].click.sum()]
statistic_8["Spend"] = budget_8.payprice.sum()/1000
statistic_8["CTR%"] = statistic_8["Click"]/budget_8.shape[0]*100
statistic_8["CPC"] = statistic_8["Spend"]/statistic_8["Click"]
statistic_8["CPM"] = statistic_8["Spend"] /budget_8.shape[0]* 1000

statistic_16=pd.DataFrame()
budget_16 = new_validation[new_validation["total_cost"]/1000<=25000*1/16]
statistic_16["Click"] = [budget_16[budget_16.click==1].click.sum()]
statistic_16["Spend"] = budget_16.payprice.sum()/1000
statistic_16["CTR%"] = statistic_16["Click"]/budget_16.shape[0]*100
statistic_16["CPC"] = statistic_16["Spend"]/statistic_16["Click"]
statistic_16["CPM"] = statistic_16["Spend"] /budget_16.shape[0]* 1000


statistic_one = statistic.append(statistic_2)
statistic_two = statistic_one.append(statistic_4)
statistic_3 = statistic_two.append(statistic_8)
statistic_combine = statistic_3.append(statistic_16)

statistic_combine.index = ["full","1/2","1/4 (6250)","1/8","1/16"]
statistic_combine

/Users/Wendy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Click,Spend,CTR%,CPC,CPM
full,119,6091.495,0.082480,51.189034,42.220832
1/2,119,6091.495,0.082480,51.189034,42.220832
1/4 (6250),119,6091.495,0.082480,51.189034,42.220832
1/8,66,3124.995,0.088876,47.348409,42.081240
1/16,38,1562.491,0.102200,41.118184,42.022780


In [29]:
test = pd.read_csv("test.csv")

In [30]:
test["area"] = test["slotheight"]*test["slotwidth"] 

city = pd.DataFrame(test.city.value_counts()/test.shape[0]).index
frequent = pd.DataFrame(test.city.value_counts()/test.shape[0]).values
r = pd.DataFrame()
r["city"] = city
r["frequent"] = frequent
test=pd.merge(test,r,on='city',how='left') 

feature = ["weekday","hour","advertiser","adexchange","area","slotvisibility","slotformat","frequent",
          "slotprice"]
X_test = test[feature]

X_test.head()

,weekday,hour,advertiser,adexchange,area,slotvisibility,slotformat,frequent,slotprice
0,0,17,3427,1,75000,1,1,0.019370,0
1,5,22,1458,3,90000,0,0,0.042239,80
2,2,13,3427,3,90000,0,0,0.004567,20
3,4,17,3386,1,94080,2,1,0.001481,0
4,6,20,3476,3,90000,0,0,0.002118,31


In [31]:
np.random.seed(12)

# Initialize label encoder
label_encoder = preprocessing.LabelEncoder()

X_test["slotvisibility"] = label_encoder.fit_transform(X_test["slotvisibility"])
X_test["slotformat"] = label_encoder.fit_transform(X_test["slotformat"])
X_test["adexchange"] = label_encoder.fit_transform(X_test["adexchange"])


X_test["advertiser"]=X_test.advertiser.astype("str")
X_test["weekday"]=X_test.weekday.astype("str")
X_test["hour"]=X_test.hour.astype("str")

X_test=pd.get_dummies(X_test)

/Users/Wendy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Wendy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Wendy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/in

In [32]:
Y_pred_test_gbr = GBR.predict_proba(X_test)
Y_pred_test_gbr = Y_pred_test_gbr [:,1]


In [33]:
test["pCTR_gbr"] = Y_pred_test_gbr

avgCTR = Y_pred_test_gbr.mean()

param_test_gbr= Y_pred_test_gbr/avgCTR

test['new_bidprice_gbr'] = 70*param_test_gbr

In [34]:
test["area"] = test["slotheight"]*test["slotwidth"] 

feature = ["weekday","hour","advertiser","adexchange","area","slotvisibility","slotformat",
          "slotprice"]
X_test = test[feature]
X_test.head()

,weekday,hour,advertiser,adexchange,area,slotvisibility,slotformat,slotprice
0,0,17,3427,1,75000,1,1,0
1,5,22,1458,3,90000,0,0,80
2,2,13,3427,3,90000,0,0,20
3,4,17,3386,1,94080,2,1,0
4,6,20,3476,3,90000,0,0,31


In [35]:
X_test.loc[ X_test['slotprice'] <= 0, 'slotprice'] = 0
X_test.loc[(X_test['slotprice'] > 0) & (X_test['slotprice'] <= 10), 'slotprice'] = 1
X_test.loc[(X_test['slotprice'] > 10) & (X_test['slotprice'] <= 50), 'slotprice'] = 2
X_test.loc[(X_test['slotprice'] > 50) & (X_test['slotprice'] <= 100), 'slotprice'] = 3
X_test.loc[(X_test['slotprice'] > 100), 'slotprice'] = 4

X_test.area.value_counts().sort_index()
X_test.loc[ X_test['area'] <= 20000, 'area'] = 0
X_test.loc[(X_test['area'] > 20000) & (X_test['area'] <=40000 ), 'area'] = 1
X_test.loc[(X_test['area'] > 40000) & (X_test['area'] <= 60000), 'area'] = 2
X_test.loc[(X_test['area'] > 60000) & (X_test['area'] <= 80000), 'area'] = 3
X_test.loc[(X_test['area'] > 80000) & (X_test['area'] <= 100000), 'area'] = 4
X_test.loc[(X_test['area'] > 100000), 'area'] = 5


/Users/Wendy/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [36]:
X_test["advertiser"]=X_test.advertiser.astype("str")
X_test["weekday"]=X_test.weekday.astype("str")
X_test["adexchange"]=X_test.adexchange.astype("str")
X_test["slotvisibility"]=X_test.slotvisibility.astype("str")
X_test["slotformat"]=X_test.slotformat.astype("str")
X_test["hour"]=X_test.hour.astype("str")

# X_test["slotprice"]=X_test.slotprice.astype("str")
# X_test["area"]=X_test.area.astype("str")

X_test=pd.get_dummies(X_test)

/Users/Wendy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/Wendy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/Wendy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [37]:
Y_pred_test = logreg.predict_proba(X_test)
Y_pred_test_log = Y_pred_test[:,1]
Y_pred_test_log

array([ 0.00127724,  0.00106514,  0.00075907, ...,  0.00125908,
        0.00040414,  0.00021689])

In [38]:
test["pCTR_log"] = Y_pred_test_log

# avgCTR = (train[train.click==1].groupby("advertiser").size().values/train.groupby("advertiser").size().values).mean()
avgCTR = Y_pred_test_log.mean()

param_test_log= Y_pred_test_log/avgCTR
param_test_log

array([ 1.68980895,  1.4092034 ,  1.00426732, ...,  1.6657838 ,
        0.53468387,  0.28695043])

In [39]:
test['new_bidprice_log'] = 70*param_validation_log

Y_pred_test_combine = 0.1 * Y_pred_test_log + 0.9 * Y_pred_test_gbr
test["pCTR_combine"] = Y_pred_test_combine
avg_ctr = Y_pred_test_combine.mean()
param_test_combine = Y_pred_test_combine/avg_ctr

test["new_bidprice_combine"] = 70 * param_test_combine


In [40]:
output = test [["bidid","new_bidprice_combine"]]
output.columns = ["bidid","bidprice"]
output.head()

,bidid,bidprice
0,fe2e06dff1dfd227471fd1ca717888ac,99.447123
1,92ea762b7b2cfd7aee30a2e88e1f5899,87.653608
2,617dfab714eedf4988c4aa9ad5425eca,46.150101
3,7fa016aaa3db39413b482f757facc3f4,53.683457
4,49db062ac10d19591dc7d988d730aad5,37.292951


In [42]:
output.to_csv("testing bidding price.csv")